Подготовка датасета для обучения

In [318]:
import pandas as pd
df_dataset=pd.read_csv("./live_dat.csv",low_memory=False)


In [319]:
df_dataset["Label"].value_counts()

0    1058
1    1001
2     180
Name: Label, dtype: int64

In [320]:
import numpy as np

In [321]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Предобработка

In [322]:
train, test = train_test_split(df_dataset, test_size=0.3, random_state=420)

In [323]:
numerical_cols = df_dataset.columns[:-3]

In [324]:
min_max_scaler = MinMaxScaler().fit(train[numerical_cols])

In [325]:
train[numerical_cols] = min_max_scaler.transform(train[numerical_cols])

In [326]:
train.shape

(1567, 72)

In [327]:
test[numerical_cols] = min_max_scaler.transform(test[numerical_cols])

In [328]:
test

,protocol,flow_duration,fwd_pkts_s,bwd_pkts_s,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,...,fwd_blk_rate_avg,bwd_blk_rate_avg,fwd_seg_size_avg,bwd_seg_size_avg,cwe_flag_count,subflow_fwd_pkts,subflow_bwd_pkts,subflow_fwd_byts,subflow_bwd_byts,Label
76,1.0,0.975115,1.200519e-05,0.000000e+00,0.259942,0.000000,0.344702,0.000000,0.111111,1.000000,...,0.000198,0.0,0.551185,0.000000,0.0,0.259942,0.000000,2178646,0,2
522,0.0,0.156173,2.604520e-07,2.083616e-07,0.000723,0.000317,0.000119,0.000014,0.039346,0.000000,...,0.000000,0.0,0.040546,0.016363,0.0,0.000723,0.000317,807,240,0
1729,0.0,0.013017,4.374874e-06,3.124910e-06,0.001085,0.000396,0.000143,0.000019,0.013090,0.008219,...,0.000000,0.0,0.031281,0.018544,0.0,0.001085,0.000396,958,340,1
781,1.0,0.000042,3.862495e-04,3.862495e-04,0.000181,0.000159,0.000016,0.000061,0.001903,0.017123,...,0.000000,0.0,0.009438,0.146173,0.0,0.000181,0.000159,158,1072,0
1290,0.0,0.000007,4.767580e-03,5.959476e-03,0.000542,0.000396,0.000060,0.000019,0.013090,0.008219,...,0.000000,0.0,0.020386,0.017890,0.0,0.000542,0.000396,432,328,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,0.0,0.175852,2.775670e-07,2.313058e-07,0.000904,0.000396,0.000128,0.000017,0.039346,0.000000,...,0.000000,0.0,0.033788,0.016363,0.0,0.000904,0.000396,861,300,0
1655,0.0,0.000016,2.053388e-03,2.566735e-03,0.000542,0.000396,0.000060,0.000019,0.013090,0.008219,...,0.000000,0.0,0.020386,0.017890,0.0,0.000542,0.000396,432,328,1
1606,0.0,0.000015,2.169197e-03,2.711497e-03,0.000542,0.000396,0.000060,0.000019,0.013090,0.008219,...,0.000000,0.0,0.020386,0.018544,0.0,0.000542,0.000396,432,340,1
307,0.0,0.000199,1.227345e-04,1.227345e-04,0.000362,0.000238,0.000102,0.000010,0.039346,0.000000,...,0.000000,0.0,0.067577,0.016363,0.0,0.000362,0.000238,699,180,0


In [329]:
y_train = np.array(train.pop("Label")) # pop removes "Label" from the dataframe
X_train = train.values

In [330]:
y_train

array([2, 1, 0, ..., 0, 0, 0])

In [331]:
y_test = np.array(test.pop("Label")) # pop removes "Label" from the dataframe
X_test = test.values

Ансамбл SVM + MLP выход VOTE. Тестим смешивание

In [332]:
from numpy import hstack
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [333]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.utils import class_weight

In [334]:
models = {}
label= {0 :'BENIGN',
    1: 'http',
    2: 'udp',
}
np.unique(y_train)
len(y_train)

1567

Родбор параметров моделей

Рандомный лес

In [335]:
# Calculating class weights for balanced class weighted classifier training
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

print( [i for i,j in label.items()])

# Must be in dict format for scikitlearn
class_weights = {
    i: class_weights[i] for i,j in label.items()
    #1: class_weights[1],
    #2: class_weights[2],
    #3: class_weights[3]
}

print(class_weights)

[0, 1, 2]
{0: 0.6964444444444444, 1: 0.7526416906820365, 2: 4.246612466124661}


In [336]:
class_weights

{0: 0.6964444444444444, 1: 0.7526416906820365, 2: 4.246612466124661}

In [337]:
model = RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=class_weights,
    ccp_alpha=0.0,
    max_samples=None
)

hyperparameters = {
    'n_estimators': [75,100, 125, 150,175]
}

In [338]:
X_train

array([[1.00000000e+00, 9.48424361e-01, 1.17340431e-05, ...,
        0.00000000e+00, 2.07115200e+06, 0.00000000e+00],
       [0.00000000e+00, 1.48718238e-04, 2.18806411e-04, ...,
        3.96290719e-04, 4.32000000e+02, 3.40000000e+02],
       [1.00000000e+00, 7.23660134e-04, 6.74498342e-05, ...,
        6.34065150e-04, 1.94600000e+03, 7.36100000e+03],
       ...,
       [0.00000000e+00, 1.56372652e-01, 2.60119846e-07, ...,
        3.17032575e-04, 8.07000000e+02, 2.40000000e+02],
       [0.00000000e+00, 5.26275212e-02, 7.72896558e-07, ...,
        4.75548863e-04, 3.72000000e+02, 4.50000000e+02],
       [0.00000000e+00, 8.89688863e-02, 3.45086545e-04, ...,
        5.76206705e-01, 2.76986000e+05, 9.37606500e+06]])

In [339]:
rf = GridSearchCV(
    estimator=model,
    param_grid=hyperparameters,
    cv=2,
    verbose=1,
    n_jobs=8  # Use all available CPU cores
)

In [340]:
rf.fit(X=X_train, y=y_train)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


GridSearchCV(cv=2,
             estimator=RandomForestClassifier(class_weight={0: 0.6964444444444444,
                                                            1: 0.7526416906820365,
                                                            2: 4.246612466124661}),
             n_jobs=8, param_grid={'n_estimators': [75, 100, 125, 150, 175]},
             verbose=1)

In [341]:
models['random_forest'] = rf.best_estimator_
n_estimator=rf.best_params_
rf.score(X=X_train, y=y_train)
n_estimator

{'n_estimators': 75}

Дерево решений

In [342]:
from sklearn.tree import DecisionTreeClassifier

In [343]:
model = DecisionTreeClassifier(
    criterion='gini',
    splitter='best',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    random_state=None,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    class_weight=None,
    ccp_alpha=0.0
)

hyperparameters = {
    'max_depth': [i for i in range(10, 20)]
}

In [344]:
d_tree = GridSearchCV(
    estimator=model,
    param_grid=hyperparameters,
    cv=2,
    verbose=1,
    n_jobs=8  # Use all available CPU cores
)

In [345]:
d_tree.fit(X=X_train, y=y_train)

Fitting 2 folds for each of 10 candidates, totalling 20 fits


GridSearchCV(cv=2, estimator=DecisionTreeClassifier(), n_jobs=8,
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19]},
             verbose=1)

In [346]:
models['d_tree'] = d_tree.best_estimator_
depth=d_tree.best_estimator_
depth


DecisionTreeClassifier(max_depth=15)

Ансамбль стекинг

В зоде побора было получено, что лучшие значения для моделей ниже:

In [347]:
rf = RandomForestClassifier(class_weight=class_weights,
                            n_estimators=75)

d_tree = RandomForestClassifier(class_weight=class_weights,
                            n_estimators=75)

In [348]:
from sklearn.ensemble import StackingClassifier
from keras.models import save_model

In [349]:
estimators = [('rf', rf), ('dt', d_tree)]
log_model = StackingClassifier(estimators=estimators)

In [350]:
log_model.fit(X_train, y_train).score(X_test, y_test)


1.0

In [351]:
y_pred = log_model.predict(X_test)

In [352]:
import pickle
arquivo = 'Models/'+ 'log' + '.pkl'
with open(arquivo,'wb') as file:
    pickle.dump(log_model,file)
    print('Model sklearn saved')

Model sklearn saved


In [353]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=list(label.values())))

              precision    recall  f1-score   support

      BENIGN       1.00      1.00      1.00       308
        http       1.00      1.00      1.00       307
         udp       1.00      1.00      1.00        57

    accuracy                           1.00       672
   macro avg       1.00      1.00      1.00       672
weighted avg       1.00      1.00      1.00       672



Voting

In [354]:
model_voting = {}

In [355]:
model = RandomForestClassifier(
    n_estimators=125,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=class_weights,
    ccp_alpha=0.0,
    max_samples=None
)

In [356]:
model_voting['random_forest'] = model

In [357]:
model =RandomForestClassifier(
    n_estimators=150,
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    class_weight=class_weights,
    ccp_alpha=0.0,
    max_samples=None
)
# model = DecisionTreeClassifier(
    # criterion='gini',
    # splitter='best',
    # max_depth=19,
    # min_samples_split=2,
    # min_samples_leaf=1,
    # min_weight_fraction_leaf=0.0,
    # max_features=None,
    # random_state=None,
    # max_leaf_nodes=None,
    # min_impurity_decrease=0.0,
    # class_weight=None,
    # ccp_alpha=0.0
# )

In [358]:
model_voting['d_tree'] = model

In [359]:
estimators = [('rf', model_voting['random_forest']), ('dt', model_voting['d_tree'])]

In [360]:
voting_model = VotingClassifier(estimators=estimators, voting='hard')

In [361]:
voting_model.fit(X_train, y_train).score(X_test, y_test)

1.0

In [362]:
y_pred = voting_model.predict(X_test)

In [363]:
from sklearn.metrics import classification_report

In [364]:
print(classification_report(y_test, y_pred, target_names=list(label.values())))

              precision    recall  f1-score   support

      BENIGN       1.00      1.00      1.00       308
        http       1.00      1.00      1.00       307
         udp       1.00      1.00      1.00        57

    accuracy                           1.00       672
   macro avg       1.00      1.00      1.00       672
weighted avg       1.00      1.00      1.00       672



In [365]:
from keras.models import Sequential
from keras.layers import Dense,GRU,Embedding,Dropout,Flatten,Conv1D,MaxPooling1D,add,GlobalAveragePooling1D,LSTM
from keras import optimizers,losses,utils,Model,Input
from tensorflow_addons.metrics import F1Score

for i,j in enumerate(X_train):
    X_train[i]=np.array(j)
for i,j in enumerate(X_test):
    X_test[i]=np.array(j)


In [366]:
def GRU_model():

    model = Sequential()
    #model.add(Embedding())
    
    model.add(GRU(128,input_shape=(71,1), return_sequences=True)) #
    #model.add(GRU(64)) 
    model.add(GRU(32))
    
    #model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    # model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))    
    model.add(Dense(3, activation='softmax'))
    
    model.build()
    print(model.summary())
    
    return model
Y_train=[]
for i in y_train:
    j=np.zeros(3)
    j[i]=1
    Y_train.append(j)
print(y_train[0])
Y_train=np.array(Y_train)
print(Y_train[0])
Y_train.shape
Y_test=[]
for i in y_test:
    j=np.zeros(3)
    j[i]=1
    Y_test.append(j)
print(y_train[0:2])
Y_test=np.array(Y_test)
Y_test.shape

def model1():
    model = Sequential([
    Conv1D(128, 16, padding='same', activation='relu', input_shape=(71,
    1)),
    MaxPooling1D(2),
    Conv1D(64, 16, padding='same', activation='relu'),
    MaxPooling1D(2),
    Conv1D(32, 8, padding='same', activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
    ])
    return model

def model2():
    start = Input(shape=(71,1),name="a")

    x=Conv1D(128,8,activation='relu')(start)
    x=Conv1D(64,4,activation='relu')(x)
    x=Conv1D(64,4,activation='relu',padding='same')(x)
    inp = MaxPooling1D(2)(x)

    x=Conv1D(128,8,activation='relu',padding='same')(inp)
    x=Conv1D(64,4,activation='relu',padding='same')(x)
    x=Conv1D(64,4,activation='relu',padding='same')(x)

    new = add([x,inp])

    x=Conv1D(128,8,activation='relu',padding='same')(new)
    x=Conv1D(64,4,activation='relu',padding='same')(x)
    x=Conv1D(64,4,activation='relu',padding='same')(x)
    
    last=add([x,new])


    x=Conv1D(128,8,activation='relu',padding='same')(last)
    x=GlobalAveragePooling1D()(x)
    x=Dense(128,activation='relu')(x)
    #x=GRU(128,8)(x)
    x=Dropout(0.5)(x)
    out=Dense(3,activation='softmax')(x)

    model=Model(start,out,name='NewNet')
    return model
def lstm_model():

    model = Sequential()
    #model.add(Embedding())
    
    model.add(LSTM(128,input_shape=(71,1), return_sequences=True)) #
    #model.add(GRU(64)) 
    model.add(LSTM(32))
    
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))    
    model.add(Dense(3, activation='softmax'))
    
    model.build()
    print(model.summary())
    
    return model


2
[0. 0. 1.]
[2 1]


In [367]:
from keras.utils import to_categorical
model=model1()
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss=losses.categorical_crossentropy,
    metrics=['accuracy', F1Score(3)]
)
model.fit(X_train,Y_train, batch_size=32, epochs=20)


mod_pred=np.argmax(model.predict(X_test), axis=1)
mod_pred_categorical = to_categorical(mod_pred)

Epoch 1/20
49/49 [==============================] - 1s 9ms/step - loss: 592.0443 - accuracy: 0.4410 - f1_score: 0.4062
Epoch 2/20
49/49 [==============================] - 0s 9ms/step - loss: 282.3249 - accuracy: 0.6586 - f1_score: 0.6531
Epoch 3/20
49/49 [==============================] - 0s 9ms/step - loss: 324.7858 - accuracy: 0.6203 - f1_score: 0.6645
Epoch 4/20
49/49 [==============================] - 0s 9ms/step - loss: 172.2332 - accuracy: 0.7958 - f1_score: 0.8220
Epoch 5/20
49/49 [==============================] - 0s 9ms/step - loss: 114.1373 - accuracy: 0.7830 - f1_score: 0.8059
Epoch 6/20
49/49 [==============================] - 0s 9ms/step - loss: 101.6713 - accuracy: 0.8845 - f1_score: 0.8884
Epoch 7/20
49/49 [==============================] - 0s 8ms/step - loss: 116.5168 - accuracy: 0.9496 - f1_score: 0.9366
Epoch 8/20
49/49 [==============================] - 0s 8ms/step - loss: 68.9904 - accuracy: 0.9317 - f1_score: 0.9201
Epoch 9/20
49/49 [==============================]

In [368]:
model.summary()

Model: "sequential_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_300 (Conv1D)         (None, 71, 128)           2176      
                                                                 
 max_pooling1d_57 (MaxPoolin  (None, 35, 128)          0         
 g1D)                                                            
                                                                 
 conv1d_301 (Conv1D)         (None, 35, 64)            131136    
                                                                 
 max_pooling1d_58 (MaxPoolin  (None, 17, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_302 (Conv1D)         (None, 17, 32)            16416     
                                                                 
 max_pooling1d_59 (MaxPoolin  (None, 8, 32)          

In [369]:
print(classification_report(Y_test, mod_pred_categorical, target_names=list(label.values())))

              precision    recall  f1-score   support

      BENIGN       0.99      0.99      0.99       308
        http       0.99      0.99      0.99       307
         udp       0.98      1.00      0.99        57

   micro avg       0.99      0.99      0.99       672
   macro avg       0.99      0.99      0.99       672
weighted avg       0.99      0.99      0.99       672
 samples avg       0.99      0.99      0.99       672



In [370]:
model=model2()
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss=losses.categorical_crossentropy,
    metrics=['accuracy', F1Score(3)]
)
model.fit(X_train,Y_train, batch_size=32, epochs=20)


mod_pred=np.argmax(model.predict(X_test), axis=1)
mod_pred_categorical = to_categorical(mod_pred)




Epoch 1/20
49/49 [==============================] - 1s 13ms/step - loss: 28.3930 - accuracy: 0.4505 - f1_score: 0.3818
Epoch 2/20
49/49 [==============================] - 1s 13ms/step - loss: 2.5681 - accuracy: 0.5846 - f1_score: 0.6662
Epoch 3/20
49/49 [==============================] - 1s 14ms/step - loss: 3.1599 - accuracy: 0.6037 - f1_score: 0.6874
Epoch 4/20
49/49 [==============================] - 1s 15ms/step - loss: 1.4641 - accuracy: 0.6299 - f1_score: 0.7103
Epoch 5/20
49/49 [==============================] - 1s 15ms/step - loss: 1.4578 - accuracy: 0.7888 - f1_score: 0.8240
Epoch 6/20
49/49 [==============================] - 1s 13ms/step - loss: 1.7959 - accuracy: 0.8366 - f1_score: 0.8619
Epoch 7/20
49/49 [==============================] - 1s 14ms/step - loss: 0.7948 - accuracy: 0.8775 - f1_score: 0.8830
Epoch 8/20
49/49 [==============================] - 1s 13ms/step - loss: 1.0427 - accuracy: 0.9017 - f1_score: 0.9133
Epoch 9/20
49/49 [==============================] - 1s 

In [371]:
model.summary()

Model: "NewNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 a (InputLayer)                 [(None, 71, 1)]      0           []                               
                                                                                                  
 conv1d_303 (Conv1D)            (None, 64, 128)      1152        ['a[0][0]']                      
                                                                                                  
 conv1d_304 (Conv1D)            (None, 61, 64)       32832       ['conv1d_303[0][0]']             
                                                                                                  
 conv1d_305 (Conv1D)            (None, 61, 64)       16448       ['conv1d_304[0][0]']             
                                                                                             

In [372]:
print(classification_report(Y_test, mod_pred_categorical, target_names=list(label.values())))

              precision    recall  f1-score   support

      BENIGN       1.00      0.94      0.97       308
        http       0.95      1.00      0.97       307
         udp       1.00      1.00      1.00        57

   micro avg       0.97      0.97      0.97       672
   macro avg       0.98      0.98      0.98       672
weighted avg       0.98      0.97      0.97       672
 samples avg       0.97      0.97      0.97       672



In [373]:
model=GRU_model()
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss=losses.categorical_crossentropy,
    metrics=['accuracy', F1Score(3)]
)
model.summary()
model.fit(X_train,Y_train, batch_size=32, epochs=20)


mod_pred=np.argmax(model.predict(X_test), axis=1)
mod_pred_categorical = to_categorical(mod_pred)

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_26 (GRU)                (None, 71, 128)           50304     
                                                                 
 gru_27 (GRU)                (None, 32)                15552     
                                                                 
 dense_179 (Dense)           (None, 128)               4224      
                                                                 
 dropout_56 (Dropout)        (None, 128)               0         
                                                                 
 dense_180 (Dense)           (None, 3)                 387       
                                                                 
Total params: 70,467
Trainable params: 70,467
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_39"
_____________________

In [374]:
model.summary()

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_26 (GRU)                (None, 71, 128)           50304     
                                                                 
 gru_27 (GRU)                (None, 32)                15552     
                                                                 
 dense_179 (Dense)           (None, 128)               4224      
                                                                 
 dropout_56 (Dropout)        (None, 128)               0         
                                                                 
 dense_180 (Dense)           (None, 3)                 387       
                                                                 
Total params: 70,467
Trainable params: 70,467
Non-trainable params: 0
_________________________________________________________________


In [375]:
print(classification_report(Y_test, mod_pred_categorical, target_names=list(label.values())))

              precision    recall  f1-score   support

      BENIGN       1.00      0.99      1.00       308
        http       1.00      1.00      1.00       307
         udp       0.97      1.00      0.98        57

   micro avg       1.00      1.00      1.00       672
   macro avg       0.99      1.00      0.99       672
weighted avg       1.00      1.00      1.00       672
 samples avg       1.00      1.00      1.00       672



In [376]:
model=lstm_model()
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss=losses.categorical_crossentropy,
    metrics=['accuracy', F1Score(3)]
)
model.summary()
model.fit(X_train,Y_train, batch_size=32, epochs=20)


mod_pred=np.argmax(model.predict(X_test), axis=1)
mod_pred_categorical = to_categorical(mod_pred)

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_30 (LSTM)              (None, 71, 128)           66560     
                                                                 
 lstm_31 (LSTM)              (None, 32)                20608     
                                                                 
 dense_181 (Dense)           (None, 128)               4224      
                                                                 
 dense_182 (Dense)           (None, 128)               16512     
                                                                 
 dense_183 (Dense)           (None, 64)                8256      
                                                                 
 dropout_57 (Dropout)        (None, 64)                0         
                                                                 
 dense_184 (Dense)           (None, 3)               

In [377]:
model.summary()

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_30 (LSTM)              (None, 71, 128)           66560     
                                                                 
 lstm_31 (LSTM)              (None, 32)                20608     
                                                                 
 dense_181 (Dense)           (None, 128)               4224      
                                                                 
 dense_182 (Dense)           (None, 128)               16512     
                                                                 
 dense_183 (Dense)           (None, 64)                8256      
                                                                 
 dropout_57 (Dropout)        (None, 64)                0         
                                                                 
 dense_184 (Dense)           (None, 3)               

In [378]:
print(classification_report(Y_test, mod_pred_categorical, target_names=list(label.values())))

              precision    recall  f1-score   support

      BENIGN       1.00      0.99      1.00       308
        http       1.00      1.00      1.00       307
         udp       0.97      1.00      0.98        57

   micro avg       1.00      1.00      1.00       672
   macro avg       0.99      1.00      0.99       672
weighted avg       1.00      1.00      1.00       672
 samples avg       1.00      1.00      1.00       672



In [395]:
from keras.layers import concatenate

def ensemble_model():
    input_layer = Input(shape=(71,1))
    # модель GRU
    gru_model = GRU_model()(input_layer)

    # модель model1
    model_1 = model1()(input_layer)

    # модель model2
    #model_2 = model2()(input_layer)

    # модель lstm
    lstm = lstm_model()(input_layer)

    # Объединение выходов моделей
    merge = concatenate([model_1,gru_model, lstm])

    # Полносвязный слой
    dense = Dense(128, activation='relu')(merge)

    # Выходной слой
    output_layer = Dense(3, activation='softmax')(dense)

    # Создание модели-ансамбля
    ensemble = Model(inputs=input_layer, outputs=output_layer, name='Ensemble')

    return ensemble
model = ensemble_model()


Model: "sequential_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_38 (GRU)                (None, 71, 128)           50304     
                                                                 
 gru_39 (GRU)                (None, 32)                15552     
                                                                 
 dense_240 (Dense)           (None, 128)               4224      
                                                                 
 dropout_73 (Dropout)        (None, 128)               0         
                                                                 
 dense_241 (Dense)           (None, 3)                 387       
                                                                 
Total params: 70,467
Trainable params: 70,467
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_58"
_____________________

In [396]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss=losses.categorical_crossentropy,
    metrics=['accuracy', F1Score(3)]
)
model.summary()
model.fit(X_train,Y_train, batch_size=32, epochs=20)


mod_pred=np.argmax(model.predict(X_test), axis=1)
mod_pred_categorical = to_categorical(mod_pred)

Model: "Ensemble"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 71, 1)]      0           []                               
                                                                                                  
 sequential_57 (Sequential)     (None, 3)            166371      ['input_16[0][0]']               
                                                                                                  
 sequential_56 (Sequential)     (None, 3)            70467       ['input_16[0][0]']               
                                                                                                  
 sequential_58 (Sequential)     (None, 3)            116355      ['input_16[0][0]']               
                                                                                           

In [397]:
model.summary()

Model: "Ensemble"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 71, 1)]      0           []                               
                                                                                                  
 sequential_57 (Sequential)     (None, 3)            166371      ['input_16[0][0]']               
                                                                                                  
 sequential_56 (Sequential)     (None, 3)            70467       ['input_16[0][0]']               
                                                                                                  
 sequential_58 (Sequential)     (None, 3)            116355      ['input_16[0][0]']               
                                                                                           

In [398]:
print(classification_report(Y_test, mod_pred_categorical, target_names=list(label.values())))

              precision    recall  f1-score   support

      BENIGN       0.99      0.99      0.99       308
        http       1.00      0.99      0.99       307
         udp       0.93      1.00      0.97        57

   micro avg       0.99      0.99      0.99       672
   macro avg       0.97      0.99      0.98       672
weighted avg       0.99      0.99      0.99       672
 samples avg       0.99      0.99      0.99       672

